**Important:**
> If you are inspecting this notebook through Github, you should hover to the upper right part of the notebook (over theta symbol) and click on *external view available with nbviewer*. This is necessary if you want to see charts on Github.

---
# Exploratory Data Analysis
---
DataVis Supplementary Material

In [1]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn import datasets

In [2]:
# Uncomment if you are using dark jupyter lab/notebook theme
#alt.renderers.set_embed_options(theme='dark')

---
## Data import

In [3]:
data_iris = datasets.load_iris (as_frame = True).frame
data_diabetes = datasets.load_diabetes (as_frame = True).frame
data_wine = datasets.load_wine (as_frame = True).frame

In [4]:
data_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
data_diabetes.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [6]:
data_wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


---
## Correlation matrix chart

In [7]:
def create_corr_chart (data):
    
    # Removing target column
    new_data = data.drop ('target', axis = 1)
    corr = new_data.corr().reset_index().melt('index')
    corr.columns = ['var_1', 'var_2', 'correlation']
    
    # Create correlation matrix chart
    chart = alt.Chart(corr).mark_rect().encode(
        alt.X ('var_1', title = None, axis = alt.Axis(labelAngle = -45)),
        alt.Y ('var_2', title = None),
        alt.Color('correlation', legend = None, scale = alt.Scale(scheme = 'redblue', reverse = True)),
    ).properties(
        width = alt.Step(40),
        height = alt.Step(40)
    )
    
    # Create text values for each colored element on top of existing chart
    chart += chart.mark_text(size = 12).encode(
        alt.Text ('correlation', format = ".2f"),
        color = alt.condition("abs(datum.correlation) > 0.5", alt.value('white'), alt.value('black'))
    )
    
    # If we want to return the chart with upper triangle as well, we should remove .transform...
    return chart.transform_filter("datum.var_1 < datum.var_2")
    

In [8]:
chart_corr_wine = create_corr_chart (data_wine)
chart_corr_wine
#chart_corr_diabetes = create_corr_chart (data_diabetes)
#chart_corr_diabetes


alt.LayerChart(...)

---
## Parallel coordinates

In [9]:
def create_parallel_chart (data):
    
    new_data = data.reset_index().melt(id_vars = ['index', 'target'])
    
    chart = alt.Chart(new_data).transform_window(
        count = 'count()'
    ).mark_line().encode(
        alt.X ('variable:N'),
        alt.Y ('value:Q'),
        alt.Color ('target:N'),
        alt.Detail ('index:N'),
        opacity = alt.value(0.4),
    ).properties(width = 1000)
    
    return chart


In [10]:
# We have to omit 'proline' and 'magnesium' columns because they skew up the resulting chart. Try without removing
chart_parallel_wine = create_parallel_chart (data_wine.drop (['proline', 'magnesium'], axis = 1))
chart_parallel_wine
#chart_parallel_iris = create_parallel_chart (data_iris)
#chart_parallel_iris

alt.Chart(...)

---
## Scatter plot matrix

In [11]:
def create_scatter_matrix (data):
    
    # We want to remove 'target' from our list of features
    features = data.columns.values[data.columns.values != 'target']
    
    chart = alt.Chart(data).mark_circle().encode(
        alt.X(alt.repeat("column"), type = 'quantitative', scale = alt.Scale (nice = True)),
        alt.Y(alt.repeat("row"), type = 'quantitative', scale = alt.Scale (nice = True)),
        color = 'target:N'
    ).properties(
        width=150,
        height=150
    ).repeat(
        row = features,
        column = features
    )#.interactive()
    
    return chart


In [12]:
#chart_scatter_wine = create_scatter_matrix (data_wine)
#chart_scatter_wine
chart_scatter_iris = create_scatter_matrix (data_iris)
chart_scatter_iris


alt.RepeatChart(...)